# Fußball als Netzwerk daten analysieren

- Fußball ist eines der am besten getrackten Spiele überhaupt. Siehe https://statsbomb.com/statsbombiq/
- Ich habe einen kleinen freien Datensatz gefunden https://github.com/statsbomb/open-data
- Auch interessant https://spielverlagerung.com/bundesliga/

In [1]:
import json
import networkx as nx
import re

## Daten importieren

In [2]:
with open('19714.json') as f:
    data = json.load(f)

## Wie sehen die Daten eigentlich aus?

In [3]:
data[123]

{'id': 'c2676078-a261-4f1a-bb66-bc8c3b71b310',
 'index': 124,
 'period': 1,
 'timestamp': '00:03:17.166',
 'minute': 3,
 'second': 17,
 'type': {'id': 30, 'name': 'Pass'},
 'possession': 7,
 'possession_team': {'id': 746, 'name': 'Manchester City WFC'},
 'play_pattern': {'id': 4, 'name': 'From Throw In'},
 'off_camera': False,
 'team': {'id': 746, 'name': 'Manchester City WFC'},
 'player': {'id': 17524, 'name': 'Jennifer Beattie'},
 'position': {'id': 5, 'name': 'Left Center Back'},
 'location': [10.0, 57.0],
 'duration': 1.752473,
 'related_events': ['1f523ab1-f91c-4bda-be6c-c2bfa431027a'],
 'pass': {'length': 24.33105,
  'angle': 0.16514868,
  'height': {'id': 1, 'name': 'Ground Pass'},
  'end_location': [34.0, 61.0],
  'outcome': {'id': 9, 'name': 'Incomplete'},
  'body_part': {'id': 40, 'name': 'Right Foot'}}}

# Spieler  
- Sie sind unsere Knoten

### Spieler Team 1

In [5]:
players1 = {}
for player in data[0]["tactics"]["lineup"]:
    players1[player["player"]["name"]] = player["position"]["name"]
players1["Drew Spence"] = "Einwechsel"
players1['Francesca Kirby'] = "Einwechsel"
players1["Erin Cuthbert"] = "Einwechsel"
players1

{'Rut Hedvig Lindahl': 'Goalkeeper',
 'Jessica Carter': 'Right Back',
 'Millie Bright': 'Right Center Back',
 'Magdalena Ericsson': 'Left Center Back',
 'Jonna Andersson': 'Left Back',
 'Sophie Ingle': 'Right Defensive Midfield',
 'Maren Nævdal Mjelde': 'Left Defensive Midfield',
 'Bethany England': 'Right Wing',
 'So-yun Ji': 'Center Attacking Midfield',
 'Karen Julia Carney': 'Left Wing',
 'Ramona Bachmann': 'Center Forward',
 'Drew Spence': 'Einwechsel',
 'Francesca Kirby': 'Einwechsel',
 'Erin Cuthbert': 'Einwechsel'}

### Spieler Team 2

In [6]:
players2 = {}
for player in data[1]["tactics"]["lineup"]:
    players2[player["player"]["name"]] = player["position"]["name"]
players2['Nadia Nadim']  = "Einwechsel"
players2['Janine Beckie'] = "Einwechsel"
players2

{'Karen Louise Bardsley': 'Goalkeeper',
 'Gemma Bonner': 'Right Back',
 'Stephanie Houghton': 'Right Center Back',
 'Jennifer Beattie': 'Left Center Back',
 'Abbie McManus': 'Left Back',
 'Jill Louise Scott': 'Right Center Midfield',
 'Keira Walsh': 'Center Midfield',
 'Georgia Stanway': 'Left Center Midfield',
 'Lauren Hemp': 'Right Center Forward',
 'Nikita Parris': 'Center Forward',
 'Melissa Lawley': 'Left Center Forward',
 'Nadia Nadim': 'Einwechsel',
 'Janine Beckie': 'Einwechsel'}

## Einwechselspieler

In [7]:
list(set(players)-set(players1)-set(players2))

NameError: name 'players' is not defined

## Pässe
- Sie sind unsere Kanten

In [8]:
out = []
for r in data:
    if "pass" in r.keys():
        out.append(r)

In [10]:
len(out) #Anzahl Pässe im Spiel

947

In [11]:
out[123]

{'id': 'b26f20e7-67ee-4f4f-8f4f-99741fd63039',
 'index': 362,
 'period': 1,
 'timestamp': '00:11:35.842',
 'minute': 11,
 'second': 35,
 'type': {'id': 30, 'name': 'Pass'},
 'possession': 23,
 'possession_team': {'id': 971, 'name': 'Chelsea LFC'},
 'play_pattern': {'id': 4, 'name': 'From Throw In'},
 'off_camera': False,
 'team': {'id': 971, 'name': 'Chelsea LFC'},
 'player': {'id': 4633, 'name': 'Magdalena Ericsson'},
 'position': {'id': 5, 'name': 'Left Center Back'},
 'location': [24.0, 21.0],
 'duration': 2.184795,
 'related_events': ['6b4d4fed-a68b-4821-bec4-d5c273c740b1'],
 'pass': {'recipient': {'id': 4640, 'name': 'Rut Hedvig Lindahl'},
  'length': 26.172504,
  'angle': 2.329174,
  'height': {'id': 1, 'name': 'Ground Pass'},
  'end_location': [6.0, 40.0],
  'body_part': {'id': 38, 'name': 'Left Foot'}}}

## Pässe "zählen"

In [12]:
r = {}
players = []
for row in out:
    if "recipient" in row["pass"].keys():
        players.append(row["player"]["name"])
        players.append(row["pass"]["recipient"]["name"])
        from_player = row["player"]["name"]
        to_player = row["pass"]["recipient"]["name"]
        if from_player+","+to_player in r:
            r[from_player+","+to_player] = r[from_player+","+to_player] + 1
        else:
            r[from_player+","+to_player] = 1
players

['Lauren Hemp',
 'Georgia Stanway',
 'Georgia Stanway',
 'Stephanie Houghton',
 'Stephanie Houghton',
 'Gemma Bonner',
 'Gemma Bonner',
 'Stephanie Houghton',
 'Stephanie Houghton',
 'Jennifer Beattie',
 'Jennifer Beattie',
 'Karen Louise Bardsley',
 'Karen Louise Bardsley',
 'Stephanie Houghton',
 'Stephanie Houghton',
 'Gemma Bonner',
 'Maren Nævdal Mjelde',
 'Ramona Bachmann',
 'Ramona Bachmann',
 'Karen Julia Carney',
 'Karen Julia Carney',
 'Bethany England',
 'Jonna Andersson',
 'Ramona Bachmann',
 'Ramona Bachmann',
 'Jonna Andersson',
 'Georgia Stanway',
 'Lauren Hemp',
 'Keira Walsh',
 'Lauren Hemp',
 'Rut Hedvig Lindahl',
 'So-yun Ji',
 'So-yun Ji',
 'Bethany England',
 'Bethany England',
 'So-yun Ji',
 'So-yun Ji',
 'Sophie Ingle',
 'Sophie Ingle',
 'Ramona Bachmann',
 'Jessica Carter',
 'Rut Hedvig Lindahl',
 'Rut Hedvig Lindahl',
 'Millie Bright',
 'Millie Bright',
 'Jessica Carter',
 'Jessica Carter',
 'Millie Bright',
 'Millie Bright',
 'Rut Hedvig Lindahl',
 'Rut Hedvig

In [14]:
r

{'Lauren Hemp,Georgia Stanway': 2,
 'Georgia Stanway,Stephanie Houghton': 5,
 'Stephanie Houghton,Gemma Bonner': 24,
 'Gemma Bonner,Stephanie Houghton': 19,
 'Stephanie Houghton,Jennifer Beattie': 11,
 'Jennifer Beattie,Karen Louise Bardsley': 7,
 'Karen Louise Bardsley,Stephanie Houghton': 19,
 'Maren Nævdal Mjelde,Ramona Bachmann': 2,
 'Ramona Bachmann,Karen Julia Carney': 9,
 'Karen Julia Carney,Bethany England': 2,
 'Jonna Andersson,Ramona Bachmann': 9,
 'Ramona Bachmann,Jonna Andersson': 2,
 'Georgia Stanway,Lauren Hemp': 2,
 'Keira Walsh,Lauren Hemp': 3,
 'Rut Hedvig Lindahl,So-yun Ji': 6,
 'So-yun Ji,Bethany England': 2,
 'Bethany England,So-yun Ji': 4,
 'So-yun Ji,Sophie Ingle': 4,
 'Sophie Ingle,Ramona Bachmann': 3,
 'Jessica Carter,Rut Hedvig Lindahl': 6,
 'Rut Hedvig Lindahl,Millie Bright': 4,
 'Millie Bright,Jessica Carter': 9,
 'Jessica Carter,Millie Bright': 8,
 'Millie Bright,Rut Hedvig Lindahl': 4,
 'Millie Bright,So-yun Ji': 4,
 'So-yun Ji,Magdalena Ericsson': 2,
 'Mag

# Netzwerkdaten erzeugen

In [16]:
set(players) #set-Befehl macht eine Liste mit allen Spielern und führt sie nur einmal auf (unique)

{'Abbie McManus',
 'Bethany England',
 'Drew Spence',
 'Erin Cuthbert',
 'Francesca Kirby',
 'Gemma Bonner',
 'Georgia Stanway',
 'Janine Beckie',
 'Jennifer Beattie',
 'Jessica Carter',
 'Jill Louise Scott',
 'Jonna Andersson',
 'Karen Julia Carney',
 'Karen Louise Bardsley',
 'Keira Walsh',
 'Lauren Hemp',
 'Magdalena Ericsson',
 'Maren Nævdal Mjelde',
 'Melissa Lawley',
 'Millie Bright',
 'Nadia Nadim',
 'Nikita Parris',
 'Ramona Bachmann',
 'Rut Hedvig Lindahl',
 'So-yun Ji',
 'Sophie Ingle',
 'Stephanie Houghton'}

In [17]:
item

'Jonna Andersson,Millie Bright'

In [18]:
# Pässe zwischen den Teams werden ja als Ballverlust codiert. Pässe deshalb nur innerhalb der Teams
G  = nx.DiGraph(name="team1")
G2 = nx.DiGraph(name="team2")

for item in list(set(players)):
    item_clean = re.sub(r"[^A-Za-z0-9]+", '',item) #r = regex: Alle Buchstaben A-Z auch klein, Zahlen 0-9
    if item in players1:
        G.add_node(item_clean, size=1,label=item+" (" + players1[item] + ")", position=players1[item])
    elif item in players2:
        G2.add_node(item_clean, size=1,label=item+" (" + players2[item] + ")", position=players2[item])
j = 0
i = 0
for item,v in r.items():
    source = re.sub(r"[^A-Za-z0-9]+", '',item.split(",")[0])
    target = re.sub(r"[^A-Za-z0-9]+", '',item.split(",")[1])
    if item.split(",")[0] in players1:
        G.add_edge(source,target,weight=v) 
    elif item.split(",")[1] in players2:
        G2.add_edge(source,target,weight=v)
    else:
        print(item)

    
nx.write_gexf(G,"Team1.gexf")
nx.write_gexf(G2,"Team2.gexf")

In [ ]:
from operator import itemgetter
degc = nx.degree_centrality